<div class='bar_title'></div>

*Simulation for Decision Making (S4DM)*

# 3. Building Simulation Models with Python

Gunther Gust & Ignacio Ubeda <br>
Chair for Enterprise AI <br>
Data Driven Decisions Group <br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

<img src="images/simulation_study_steps.png" style="width:100%; float:left;" />

# Last lecture

* Introduction to Simpy
* Simpy processes as python generator functions
* Process interactions: One process calls a subsequent process
  * Battery electric vehicle example (driving and charging processes)
* Shared Resources
   * Carwash example (requesting a slot in the carwash)

# Agenda for today

* Condition events
* Example 1: Bank renege
* Custom triggering of events
* Example 2: Movie renege

Credits: The following content is adapted from the official [Simpy documentation](https://simpy.readthedocs.io/en/latest/simpy_intro/index.html) 

## Condition events

Sometimes, you want to wait for more than one event at the same time: 
* For example, you may want to wait for a resource, but __not for an unlimited amount of time__
    * E.g., customers may want to wait at most 10 minutes


* Or you may want to wait until a __set__ of events has happened
    * E.g., customers may not proceed until they have received two orders (such as food and drinks)

SimPy therefore offers the `AnyOf` and `AllOf` events which both are a Condition event.

In [1]:
import simpy
import random

In [2]:
from simpy.events import AnyOf, AllOf, Event

env = simpy.Environment()

In [3]:
def test_condition1a(env):
    t1, t2 = env.timeout(1, value='spam'), env.timeout(2, value='eggs')
    events = [t1, t2]
    ret = yield AnyOf(env, events)
    print(ret)

proc = env.process(test_condition1a(env))
env.run()


<ConditionValue {<Timeout(1, value=spam) object at 0x21d77698b80>: 'spam'}>


As a shorthand for `AnyOf` and `AllOf`, you can also use the logical operators `&` (and) and `|` (or):

In [4]:
def test_condition1b(env):
    t1, t2 = env.timeout(1, value='spam'), env.timeout(2, value='eggs')
    ret = yield t1 | t2
    print(ret)

proc = env.process(test_condition1b(env))
env.run()

<ConditionValue {<Timeout(1, value=spam) object at 0x21d77698520>: 'spam'}>


In [5]:
def test_condition2a(env):
    t1, t2 = env.timeout(1, value='spam'), env.timeout(2, value='eggs')
    events = [t1, t2]
    ret = yield AllOf(env, events)
    print(ret)

proc = env.process(test_condition2a(env))
env.run()

<ConditionValue {<Timeout(1, value=spam) object at 0x21d776c94f0>: 'spam', <Timeout(2, value=eggs) object at 0x21d776982b0>: 'eggs'}>


In [6]:
def test_condition2b(env):
    t1, t2 = env.timeout(1, value='spam'), env.timeout(2, value='eggs')
    ret = yield t1 & t2
    print(ret)

proc = env.process(test_condition2b(env))
env.run()

<ConditionValue {<Timeout(1, value=spam) object at 0x21d776c9370>: 'spam', <Timeout(2, value=eggs) object at 0x21d776c9400>: 'eggs'}>


In [7]:
def test_condition3a(env):
    e1, e2, e3 = env.timeout(1, value = 1), env.timeout(2, value = 2), env.timeout(3, value = 3)
    ret = yield (e1 & e2) | e3
    print(ret)

proc = env.process(test_condition3a(env))
env.run()

<ConditionValue {<Timeout(1, value=1) object at 0x21d7763ea90>: 1, <Timeout(2, value=2) object at 0x21d7763e070>: 2}>


`AnyOf` and `AllOf` take a list of events as an argument and are triggered when any (at least one) or all of them are triggered.

The value of a condition event is an ordered dictionary with an entry for every triggered event. 
* In the case of `AllOf`, the size of that dictionary will always be the same as the length of the event list. 
* The value dict of `AnyOf` will have at least one entry. 

In both cases, the event instances are used as keys and the event values will be the values.

The order of condition results is identical to the order in which the condition events were specified.

This allows the following idiom for conveniently fetching the values of multiple events specified in an `&` condition (including `AllOf`):

In [8]:
def fetch_values_of_multiple_events(env):
    t1, t2 = env.timeout(1, value='spam'), env.timeout(2, value='eggs')
    r1, r2 = (yield t1 & t2).values()
    print(r1, r2)

proc = env.process(fetch_values_of_multiple_events(env))
env.run()

spam eggs


__Control questions (now it's your turn):__
1. Implement `test_condition3b`, that is identical to `test_condition3a` but uses a combination of `AnyOf` and `AllOf`
2. Modify `test_condition2b` so that the value of the events are printed (analogously to `fetch_values_of_multiple_events`)
3. Make the duration of the timeout events in `test_condition1b` random (`random.random()` generates a random number between 0 and 1). Then, repeatedly execute `test_condition1b` and check which event has been triggered first. 

## Example: Bank renege

## Scenario

* Customers randomly arrive at a bank and want to be served (e.g. to withdraw money)

* The Bank has one counter with a random service time

* The customers form a queue in front of the counter

* Each customer has a certain willingness to wait. If his/her patience is exceeded, he/she reneges (leaves the queue) without being served

## Resource (bank counter):

In [9]:
class BankCounter:

    def __init__(self, env, mean_serving_time):
        self.env = env
        self.counter = simpy.Resource(env, capacity=1)
        self.mean_serving_time = mean_serving_time

    def serve_customer(self):
        #Assumption: Exponentially distributed serving time with mean = 1/mean_serving_time
        serving_time = random.expovariate(1.0 / self.mean_serving_time) 
        yield self.env.timeout(serving_time)

## Entities (customers):

In [10]:
class Customer:
    def __init__(self, env, name, patience):
        self.env = env
        self.name = name
        self.patience = patience

    def run(self, bank_counter):
        arrive = self.env.now
        print(f'{arrive:.4f} {self.name}: Here I am')
    
        with bank_counter.counter.request() as req:
                       
            # Wait for the counter or abort at the end of our tether
            results = yield req | self.env.timeout(self.patience)
            
            wait = self.env.now - arrive

            if req in results:
                # We got to the counter
                print(f'{self.env.now:.4f} {self.name}: Waited {wait:.3f}')
                yield self.env.process(bank_counter.serve_customer())
                print(f'{self.env.now:.4f} {self.name}: Finished')

            else:
                # We reneged
                print(f'{self.env.now:.4f} {self.name}: RENEGED after {wait:.3f}')

## Entity Generation

In [11]:
def customer_generator(env, number, interval, bank_counter):
    """Source generates customers randomly"""
    for i in range(number):
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        customer = Customer(env, f'Customer{(i+1):02d}', patience)
        env.process(customer.run(bank_counter))
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

## Setup of simulation

In [12]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10 # Generate new customers roughly every INTERVAL_CUSTOMERS minutes
MIN_PATIENCE = 1  # Min. customer patience (minutes)
MAX_PATIENCE = 3  # Max. customer patience (minutes)
SERVING_TIME = 12.0 # Mean serving time: SERVING_TIME minutes

random.seed(RANDOM_SEED)  # This helps to reproduce the results

env = simpy.Environment()
bank_counter = BankCounter(env, SERVING_TIME) #define resources
env.process(customer_generator(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, bank_counter))#customer generation


<Process(customer_generator) object at 0x21d777a29d0>

In [13]:
# Execute
print('Running Simulation...')
env.run()
print('... Done')

Running Simulation...
0.0000 Customer01: Here I am
0.0000 Customer01: Waited 0.000
0.2533 Customer02: Here I am
1.6997 Customer02: RENEGED after 1.446
3.8595 Customer01: Finished
13.5892 Customer03: Here I am
13.5892 Customer03: Waited 0.000
14.6806 Customer03: Finished
35.8621 Customer04: Here I am
35.8621 Customer04: Waited 0.000
36.1646 Customer05: Here I am
38.1753 Customer05: RENEGED after 2.011
38.8227 Customer04: Finished
... Done


__Control questions (now it's your turn):__
1. Reduce the arrival interval of customers, so that more customers enter the queue and renege
2. Implement a procedure that tracks the fraction of customers that have reneged. Print this number at the end of the simulation
3. Change other parameters of the simulation (e.g. the number of servers, the serving time and the patience). Evaluate the effect on the share of reneged customers

# Custom triggering of events

Simpy offers the functionality to manually generate events via `Event.event()`. Then, we can later manually trigger these events: 

* To trigger an event and mark it as successful, you can use `Event.succeed(value=None)`. You can optionally pass a value to it (e.g., the results of a computation).

* To trigger an event and mark it as failed, call `Event.fail(exception)` and pass an Exception instance to it (e.g., the exception you caught during your failed computation).

* There is also a generic way to trigger an event: `Event.trigger(event).` This will take the value and outcome (success or failure) of the event passed to it.

For now, we will just make use of `Event.succeed()`. Let's have a look at an example:

# Example 

In [70]:
class School:
    def __init__(self, env):
        self.env = env
        self.courses = env.process(self.course())
        self.class_ends = env.event()
      

    def course(self):
        print("Class starts")
        yield self.env.timeout(45)# classes for 45 minutes
        self.class_ends.succeed()# manually let the event happen
        print("Bell rings")

In [75]:
class Student:
    def __init__(self, env, school):
        self.env = env
        self.process = env.process(self.study(school))
    
    def study(self, school):
        print("Studying")
        yield school.class_ends #study (or sleep) until the class_ends event happens
        print('\o/') #celebrate!

In [76]:
env = simpy.Environment()
school = School(env)
for i in range(10):
    Student(env, school)
env.run()

Class starts
Studying
Studying
Studying
Studying
Studying
Studying
Studying
Studying
Studying
Studying
Bell rings
\o/
\o/
\o/
\o/
\o/
\o/
\o/
\o/
\o/
\o/



<img src="images/d3.png" style="width:50%; float:center;" />
